In [98]:
import numpy as np
import gpflow as gp

import gpflow.multioutput.kernels as mk
import gpflow.multioutput.features as mf


D = 1  # number of input dimensions
M = 20  # number of inducing points
L = 4  # number of latent GPs
P = 1  # number of observations = output dimensions

X = np.tile(np.linspace(0,1,M*2)[:,None],(1,D))
Y = np.tile(X.sum(1)[:,None],(1,P))

def _kern():
    return gp.kernels.Matern52(D)

W = np.random.uniform(size=(P,L))

kern = mk.SeparateMixedMok([_kern() for _ in range(L)], W)

feature_list = [gp.features.InducingPoints(X[:M,:])]
feature = mf.MixedKernelSharedMof(gpf.features.InducingPoints(X[:M,...].copy())) #works
#feature = mf.SeparateIndependentMof(feature_list)

q_mu = np.zeros((M, L))
q_sqrt = np.repeat(np.eye(M)[None, ...], L, axis=0) * 1.0

model = gp.models.SVGP(X, Y, kern, gp.likelihoods.Gaussian(), 
            feat = feature,
            minibatch_size=50,
            num_data = X.shape[0],
            num_latent=P,
            whiten=False,
          q_mu = q_mu,
          q_sqrt = q_sqrt)

print(model.num_latent)

opt = gpf.train.ScipyOptimizer()
opt.minimize(model, disp=True, maxiter=MAXITER)


DEBUG:gpflow.multioutput.features:Dispatch to Kuu(feat: MixedKernelSharedMof, kern: SeparateMixedMok) with jitter=1e-06
DEBUG:gpflow.multioutput.conditionals:conditional: MixedKernelSharedMof, SeparateMixedMok
DEBUG:gpflow.multioutput.conditionals:conditional: object, SharedIndependentMof, SeparateIndependentMok, object
DEBUG:gpflow.multioutput.features:Dispatch to Kuu(feat: MixedKernelSharedMof, kern: SeparateMixedMok) with jitter=1e-06
DEBUG:gpflow.multioutput.features:Dispatch to Kuf(feat: MixedKernelSharedMof, kern: SeparateMixedMok)


4
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 45.067660
  Number of iterations: 32
  Number of functions evaluations: 43


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 45.067660
  Number of iterations: 32
  Number of functions evaluations: 43
